In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shwet\\OneDrive\\Desktop\\text-summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shwet\\OneDrive\\Desktop\\text-summarization'

In [5]:
# data transformation
'''Raw dataset (e.g., dialogues and summaries) ko clean karke model-friendly format me convert karna — mainly tokenization, padding, truncation, etc.'''

'''Key Steps:
Load Raw Data
From previous ingestion stage (e.g., a CSV or dataset saved via datasets.load_from_disk)

Text Preprocessing (optional)
Clean unwanted characters, lowercase, remove HTML tags, etc.

Tokenizer Load
Load Hugging Face tokenizer like PegasusTokenizer.from_pretrained(...)

Tokenization

Inputs (dialogue) → input_ids, attention_mask

Labels (summary) → labels
With padding & truncation

Convert to Torch Dataset or HuggingFace Dataset format

Save Transformed Data
Save tokenized dataset for training & evaluation (e.g., with save_to_disk())

 Input:
Raw text dataset (dialogue + summary)

 Output:
Tokenized dataset with:

input_ids

attention_mask

labels

 Why It’s Important:
Transformer models can’t use raw text — they need numerical token IDs.

Ensures uniform input size.

Prepares data for direct training using HuggingFace Trainer.'''

'Key Steps:\nLoad Raw Data\nFrom previous ingestion stage (e.g., a CSV or dataset saved via datasets.load_from_disk)\n\nText Preprocessing (optional)\nClean unwanted characters, lowercase, remove HTML tags, etc.\n\nTokenizer Load\nLoad Hugging Face tokenizer like PegasusTokenizer.from_pretrained(...)\n\nTokenization\n\nInputs (dialogue) → input_ids, attention_mask\n\nLabels (summary) → labels\nWith padding & truncation\n\nConvert to Torch Dataset or HuggingFace Dataset format\n\nSave Transformed Data\nSave tokenized dataset for training & evaluation (e.g., with save_to_disk())\n\n Input:\nRaw text dataset (dialogue + summary)\n\n Output:\nTokenized dataset with:\n\ninput_ids\n\nattention_mask\n\nlabels\n\n Why It’s Important:\nTransformer models can’t use raw text — they need numerical token IDs.\n\nEnsures uniform input size.\n\nPrepares data for direct training using HuggingFace Trainer.'

In [6]:
 '''Data Transformation Stage

Yeh ingested data uthata hai from:
data_path: artifacts/data_ingestion/samsum_dataset

Fir tokenize karke transformed data save karta hai:
root_dir: artifacts/data_transformation'''

'Data Transformation Stage\n\nYeh ingested data uthata hai from:\ndata_path: artifacts/data_ingestion/samsum_dataset\n\nFir tokenize karke transformed data save karta hai:\nroot_dir: artifacts/data_transformation'

In [7]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])
        

        #DataTransformationConfig is return type
        data_transformation_config= DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name= config.tokenizer_name,

        )
        return data_transformation_config

In [11]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [12]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config = config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
    
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
    
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask' : input_encodings['attention_mask'],
            'labels' : target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [13]:
try:
    config=ConfigurationManager()
    data_transformation_config= config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-08-02 23:40:21,430:INFO:common:yaml file:config\config.yaml loaded successfully]
[2025-08-02 23:40:21,456:INFO:common:yaml file:params.yaml loaded successfully]
[2025-08-02 23:40:21,463:INFO:common:created directory at:artifacts]
[2025-08-02 23:40:21,463:INFO:common:created directory at:artifacts/data_transformation]


c:\Users\shwet\anaconda3\envs\textS\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shwet\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\shwet\anaconda3\envs\textS\lib\site-packag